In [1]:
from datasets import dog_dataset, cub_dataset, food_dataset, cub_and_dogs
from models.models_to_finetune import deit_small_patch16_224, myresnetv2_task1, myresnetv2_task2, myresnetv2_for_c_loss
import PIL
import numpy as np
from tqdm import tqdm
import torch
import torch.optim as optim
from torchvision import transforms
import config
import sys
import math
from loss import CenterLoss
from run_center_loss import train_model_with_closs
from vit.vit_pytorch.nest import NesT
import timm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from models import bilinear_model
from run import train_model

In [3]:
epochs = 50
batch_size = 128
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

In [4]:

import torch.nn as nn

test_transform=transforms.Compose([
                    transforms.Resize((448, 448)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])

transform = transforms.Compose([  #

        transforms.Resize((448, 448)),
        transforms.RandomRotation(20),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

# Load Data

train_loader, val_loader,test_loader = food_dataset(bs=batch_size, data_transform=transform, test_transform=test_transform)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = myresnetv2_task2(num_classes=251)
model = model.to(device)



path = "/home/hashmat.malik/Fall 2021/CV703 Lab/Week5/datasets/modelresnetv2448_food_dataset_best.pth.tar"
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['state_dict'])

my_list = ['head.weight', 'head.bias']
params = list(filter(lambda kv: kv[0] in my_list, model.named_parameters()))
base_params = list(filter(lambda kv: kv[0] not in my_list, model.named_parameters()))


optimizer = optim.Adam([
                {'params':  [i[1]for i in params], 'lr': 0.0001, 'betas': (0.5, 0.999)},
                {'params':  [i[1]for i in base_params], 'lr': 0.00001, 'betas': (0.5, 0.999)}])

criterion = torch.nn.CrossEntropyLoss()

train_model(epochs, train_loader, val_loader, test_loader, optimizer, criterion, model, 'resnetv2448_food_dataset', is_train=False)


Test: [ 0/92]	Time  8.705 ( 8.705)	Loss 2.0817e+00 (2.0817e+00)	Acc@1  53.91 ( 53.91)	Acc@5  75.78 ( 75.78)
Test: [ 5/92]	Time  2.970 ( 4.842)	Loss 1.5548e+00 (1.7071e+00)	Acc@1  62.50 ( 58.59)	Acc@5  85.94 ( 82.29)
Test: [10/92]	Time  3.011 ( 4.027)	Loss 1.6743e+00 (1.7525e+00)	Acc@1  58.59 ( 57.39)	Acc@5  84.38 ( 82.17)
Test: [15/92]	Time  2.986 ( 3.770)	Loss 1.7326e+00 (1.7350e+00)	Acc@1  56.25 ( 57.96)	Acc@5  81.25 ( 82.32)
Test: [20/92]	Time  3.532 ( 3.644)	Loss 1.8498e+00 (1.7313e+00)	Acc@1  60.94 ( 58.30)	Acc@5  78.91 ( 82.40)
Test: [25/92]	Time  3.040 ( 3.588)	Loss 1.5015e+00 (1.7378e+00)	Acc@1  64.06 ( 58.17)	Acc@5  82.03 ( 82.15)
Test: [30/92]	Time  2.868 ( 3.525)	Loss 1.8640e+00 (1.7286e+00)	Acc@1  57.81 ( 58.34)	Acc@5  80.47 ( 82.33)
Test: [35/92]	Time  3.462 ( 3.484)	Loss 1.5647e+00 (1.7140e+00)	Acc@1  57.03 ( 58.42)	Acc@5  83.59 ( 82.42)
Test: [40/92]	Time  3.141 ( 3.451)	Loss 1.9613e+00 (1.7255e+00)	Acc@1  47.66 ( 57.96)	Acc@5  77.34 ( 82.28)
Test: [45/92]	Time  3.569 ( 